## 📂 Project Structure

```
assignment/
├── data/                    # Data processing modules
│   ├── dataset.py          # SQuAD dataset loader
│   ├── preprocessing.py    # Tokenization & metrics
│   └── dataloader.py       # DataLoader utilities
│
├── training/                # Training infrastructure
│   ├── train.py            # Training loop
│   └── evaluate.py         # Evaluation metrics
│
├── inference/               # Deployment
│   └── predict.py          # QAPredictor class
│
├── notebooks/               # Jupyter notebooks
│   ├── 00_project_overview.ipynb      # This notebook
│   ├── 01_data_exploration.ipynb      # Dataset analysis
│   ├── 02_tokenizer_testing.ipynb     # Tokenization
│   ├── 03_data_validation.ipynb       # Preprocessing validation
│   ├── 04_model_training.ipynb        # Training workflow
│   ├── 05_evaluation_analysis.ipynb   # Evaluation
│   └── 06_deployment.ipynb            # Gradio demo
│
├── config.py                # Configuration
├── requirements.txt         # Dependencies
├── app.py                   # Standalone Gradio app
└── README.md               # Documentation
```

## 🚀 Quick Start Guide

### 1. Install Dependencies
```bash
pip install -r requirements.txt
```

### 2. Run Notebooks in Order

| Notebook | Purpose | Time Required |
|----------|---------|---------------|
| `01_data_exploration.ipynb` | Explore SQuAD dataset | 5 min |
| `02_tokenizer_testing.ipynb` | Understand BERT tokenization | 5 min |
| `03_data_validation.ipynb` | Validate preprocessing | 10 min |
| `04_model_training.ipynb` | Train BERT-QA model | 2-3 hours |
| `05_evaluation_analysis.ipynb` | Evaluate performance | 15 min |
| `06_deployment.ipynb` | Deploy interactive demo | 5 min |

### 3. Launch Deployment
```bash
python app.py
```
Access at: http://127.0.0.1:7860

## 💡 Key Concepts

### What is Extractive Question Answering?

Given a **question** and a **context** paragraph, the model extracts the exact answer span from the context.

**Example:**
```
Question: What is the capital of France?
Context:  Paris is the capital and most populous city of France.
Answer:   Paris
```

### How BERT Works for QA

1. **Input Format:** `[CLS] question [SEP] context [SEP]`
2. **BERT Encoding:** 12 transformer layers process the sequence
3. **QA Head:** Two linear layers predict start/end positions
4. **Output:** Token positions of the answer span

### Evaluation Metrics

**Exact Match (EM):**
- Binary score: 1 if prediction exactly matches ground truth (after normalization), 0 otherwise
- Strict metric for perfect answers

**F1 Score:**
- Token-level overlap between prediction and ground truth
- Partial credit for incomplete answers
- Computed as: F1 = 2 × (precision × recall) / (precision + recall)

## 📊 Dataset Overview

### SQuAD v1.1 (Stanford Question Answering Dataset)

- **Training Set:** 87,599 questions
- **Dev Set:** 10,570 questions
- **Source:** Wikipedia articles
- **Format:** JSON with titles, paragraphs, questions, and answers

### Data Statistics

- Average context length: ~120 words
- Average question length: ~10 words
- Average answer length: ~3 words
- Question types: What (40%), Who (12%), How (10%), When (8%), etc.

In [ ]:
import sys
sys.path.append('..')

import json
from pathlib import Path

# Quick check of dataset availability
train_file = Path('../archive/train-v1.1.json')
dev_file = Path('../archive/dev-v1.1.json')

print("Dataset Files:")
print("=" * 60)
print(f"Train: {'✓' if train_file.exists() else '✗'} {train_file}")
print(f"Dev:   {'✓' if dev_file.exists() else '✗'} {dev_file}")

if train_file.exists():
    with open(train_file, 'r') as f:
        train_data = json.load(f)
    
    total_questions = sum(
        len(qa['qas']) 
        for article in train_data['data'] 
        for qa in article['paragraphs']
    )
    
    print(f"\nTotal training questions: {total_questions:,}")
    print(f"Total articles: {len(train_data['data']):,}")

## 🔧 Configuration

### Model Configuration

In [ ]:
from config import Config

config = Config()

print("Model Configuration:")
print("=" * 60)
print(f"Model: {config.model.model_name}")
print(f"Max Length: {config.model.max_length}")
print(f"Stride: {config.model.stride}")

print("\nTraining Configuration:")
print("=" * 60)
print(f"Epochs: {config.training.epochs}")
print(f"Batch Size: {config.training.batch_size}")
print(f"Learning Rate: {config.training.learning_rate}")
print(f"Warmup Ratio: {config.training.warmup_ratio}")
print(f"Mixed Precision (FP16): {config.training.fp16}")

print("\nData Configuration:")
print("=" * 60)
print(f"Train File: {config.data.train_file}")
print(f"Dev File: {config.data.dev_file}")

## 🧪 Quick Test: Load and Predict (if model exists)

In [ ]:
from inference.predict import QAPredictor

model_path = '../checkpoints/best_model.pt'

if Path(model_path).exists():
    print("✓ Trained model found! Loading predictor...\n")
    predictor = QAPredictor(model_path)
    
    # Test prediction
    context = """Paris is the capital and most populous city of France. The city has an area 
    of 105 square kilometers and a population of 2,206,488 inhabitants."""
    
    question = "What is the capital of France?"
    
    result = predictor.predict(question, context)
    
    print("Sample Prediction:")
    print("=" * 60)
    print(f"Question: {question}")
    print(f"Answer: {result['answer']}")
    print(f"Confidence: {result['confidence']:.1f}%")
    print("=" * 60)
else:
    print("✗ No trained model found.")
    print("\nTo train the model, run: notebooks/04_model_training.ipynb")
    print("Expected training time: 2-3 hours on GPU")

## 📝 Notebook Descriptions

### Part 1: Data Understanding

**01_data_exploration.ipynb**
- Load and parse SQuAD JSON files
- Analyze dataset structure and statistics
- Visualize question/answer distributions
- Identify question types

**02_tokenizer_testing.ipynb**
- Understand BERT tokenization
- Learn about special tokens ([CLS], [SEP])
- Explore offset_mapping for character-to-token conversion
- Test stride handling for long contexts

### Part 2: Data Preprocessing

**03_data_validation.ipynb**
- Validate SQuADDataset implementation
- Test tokenization and answer span extraction
- Verify DataLoader batching
- Analyze answer position distributions
- Measure extraction accuracy (~95%+)

### Part 3: Model Training

**04_model_training.ipynb**
- Initialize BERT-QA model
- Configure trainer with optimizers and schedulers
- Train for 3 epochs with validation
- Save best checkpoint
- Monitor training/validation loss

### Part 4: Evaluation & Deployment

**05_evaluation_analysis.ipynb**
- Load best model checkpoint
- Compute EM and F1 on dev set
- Analyze performance by question type
- Error analysis and visualization
- Generate evaluation report

**06_deployment.ipynb**
- Create interactive Gradio interface
- Add example contexts and questions
- Launch web demo
- Export model to HuggingFace format
- Generate standalone app.py

## 🎯 Learning Objectives

By completing this project, you will learn:

### Technical Skills
1. **Data Processing**
   - Parse complex JSON datasets
   - Handle tokenization for transformer models
   - Map character positions to token indices
   - Implement sliding windows for long sequences

2. **Model Training**
   - Fine-tune pre-trained BERT models
   - Use mixed precision training (FP16)
   - Implement early stopping and checkpointing
   - Monitor training with TensorBoard

3. **Evaluation**
   - Compute standard QA metrics (EM, F1)
   - Analyze model errors
   - Compare with baselines
   - Generate evaluation reports

4. **Deployment**
   - Build interactive web interfaces
   - Create production-ready predictors
   - Export models for sharing
   - Document projects professionally

### Conceptual Understanding
- How transformers work for NLP tasks
- Extractive vs. generative QA
- Transfer learning and fine-tuning
- Evaluation methodology for NLP

## 🚦 Project Status Checklist

Use this checklist to track your progress:

### Setup
- [ ] Install dependencies (`pip install -r requirements.txt`)
- [ ] Verify dataset files exist in `archive/`
- [ ] Test imports in notebooks

### Part 1: Data Understanding
- [ ] Run 01_data_exploration.ipynb
- [ ] Run 02_tokenizer_testing.ipynb
- [ ] Understand SQuAD format and tokenization

### Part 2: Preprocessing
- [ ] Run 03_data_validation.ipynb
- [ ] Verify extraction accuracy >90%
- [ ] Test DataLoader batching

### Part 3: Training
- [ ] Run 04_model_training.ipynb
- [ ] Complete 3 epochs of training (2-3 hours)
- [ ] Save checkpoint to `checkpoints/best_model.pt`

### Part 4: Evaluation
- [ ] Run 05_evaluation_analysis.ipynb
- [ ] Achieve EM >80% and F1 >85%
- [ ] Review error analysis

### Part 5: Deployment
- [ ] Run 06_deployment.ipynb
- [ ] Test Gradio interface locally
- [ ] Generate standalone app.py

### Documentation
- [ ] Review README.md
- [ ] Understand project structure
- [ ] Document any custom modifications

## 🔗 Useful Resources

### Documentation
- [Hugging Face Transformers](https://huggingface.co/docs/transformers)
- [SQuAD Dataset](https://rajpurkar.github.io/SQuAD-explorer/)
- [BERT Paper](https://arxiv.org/abs/1810.04805)

### Tutorials
- [Question Answering with BERT](https://huggingface.co/transformers/task_summary.html#question-answering)
- [Fine-tuning BERT Tutorial](https://mccormickml.com/2019/07/22/BERT-fine-tuning/)

### Tools
- [Gradio Documentation](https://gradio.app/docs/)
- [PyTorch Documentation](https://pytorch.org/docs/stable/index.html)
- [TensorBoard Guide](https://www.tensorflow.org/tensorboard)

## 💻 Next Steps

### Immediate Actions
1. **If starting fresh:** Begin with `01_data_exploration.ipynb`
2. **If resuming:** Check which notebooks you've completed
3. **If trained:** Test deployment with `06_deployment.ipynb`

### Advanced Enhancements
- Experiment with different BERT variants (RoBERTa, ALBERT, DistilBERT)
- Try SQuAD v2.0 (includes unanswerable questions)
- Implement answer re-ranking
- Add multi-language support
- Deploy to cloud (AWS, GCP, HuggingFace Spaces)
- Create REST API with FastAPI

### Optimization Ideas
- Hyperparameter tuning (learning rate, batch size)
- Data augmentation techniques
- Ensemble multiple models
- Knowledge distillation for smaller models

---

**Happy Coding! 🚀**